## Computer Vision
### Lab 7
#### Project 2  - Tracking

In [2]:
import numpy as np
import cv2 as cv
import os
import glob
import matplotlib.pyplot as plt
from numpy.random import uniform
import pdb

### Particle filter

In [3]:
# a particle is a bopunding box, represented by the top left corner 
# and fixed width and hieght
def create_uniform_particles(x_range, y_range, N):
    particles = np.empty((N, 2))
    particles[:, 0] = uniform(x_range[0], x_range[1], size=N)
    particles[:, 1] = uniform(y_range[0], y_range[1], size=N) 
    return particles

In [4]:
# predict where the particles will be at the nex frame by applying some dynamics
# take into account velocity and some random noise

def predict(particles, velocity, std, frame, w, h):  
    N = len(particles)    
    
    noise = np.random.randn(N) * std[0]  
    for i in range(N):
        particles[i, 0] = particles[i, 0] + velocity[0] + noise[i]
        # check that the particle is not outside of the image
        if(particles[i, 0] > frame.shape[1] -w):
            particles[i, 0] = frame.shape[1] - w
        if(particles[i, 0] < 0):
            particles[i, 0] = 0
            
    noise = np.random.randn(N) * std[1]
    for i in range(N):
        particles[i, 1] = particles[i, 1] + velocity[1] + noise[i]
        # check the particle is still in the image
        if(particles[i, 1] > frame.shape[0] - h):
            particles[i, 1] = frame.shape[0] - h
        if(particles[i, 1] < 0):
            particles[i, 1] = 0 
    return particles

In [5]:
# update the weight of each particle based on how similar is to the target window
# use a simple color histogram model
# essential step: how to update the weights
def update(particles, frame, hist_roi_norm, w, h):
    particles = np.int32(particles)   
    weights = np.zeros((particles.shape[0],1))
    for i in range(particles.shape[0]):
        img_particle = frame[particles[i, 1]: particles[i, 1] + h - 1, particles[i, 0]:particles[i, 0] + w - 1].copy()
        particle_hist = cv.calcHist([img_particle], [0, 1, 2], None, [8, 8, 8], [0, 256, 0, 256, 0, 256]) 
        particle_hist_norm = particle_hist/ particle_hist.sum()
        distance = cv.compareHist(hist_roi_norm, particle_hist_norm, cv.HISTCMP_CHISQR_ALT)
        # higher chi-square distance is bad, smaller chi-square distance is better
        weights[i] = 1 / (np.power(distance,5))
    weights += 1.e-10 # avoid round-off to zero
    # normalize the wights such that we have a probability distribution
    weights /= sum(weights)
    return weights

In [6]:
# estimate the center of the cloud of particles
def estimate(particles, weights):    
    mean = np.float64(np.array([0,0]))
    N = particles.shape[0]  
    for i in range(N): 
        mean += weights[i] * particles[i, :]   
    return mean

In [7]:
# resample particles based on their weight
def resample(weights):
    w = weights.flatten()
    N = len(w)    
    tries = np.random.multinomial(N, w) 
    indexes = np.zeros(N, 'i')
    cumsum_vector = np.cumsum(tries)
    pos = -1 
    for i in range(len(tries)):
        for j in range(tries[i]):            
            pos = pos + 1
            indexes[pos] = i
            
    return indexes

def resample_from_index(particles, weights, indexes):
    particles[:] = particles[indexes]
    weights[:] = weights[indexes]
    weights /= np.sum(weights)
    return particles, weights

In [8]:
# change the path to match on your machine
base_folder = 'videos'
path_video = os.path.join(base_folder, "19.mp4")

cap = cv.VideoCapture(path_video)  
current_frame = 0
max_number_of_frame_to_run = 750

ret, frame = cap.read() # Read the frame
frame_gray = cv.cvtColor(frame,cv.COLOR_BGR2GRAY)
old_frame_gray = frame_gray

ret, first_frame = cap.read() # Read the first frame         
x, y, w, h = cv.selectROI(first_frame) 
img_roi = frame[y: y + h, x: x + w].copy()
hist_roi = cv.calcHist([img_roi], [0 ,1, 2], None, [8, 8, 8], [0, 256, 0, 256, 0, 256]) 
hist_roi_norm = hist_roi/ hist_roi.sum()
bb = np.array([x, y, x + w - 1, y + h - 1])

N = 1000
particles = create_uniform_particles([x, x], [y, y], N)
velocity = [0, 0]
std = [10, 10] 

while(cap.isOpened()): 
    ret, frame = cap.read() # Read the frame   
    if ret is True: 
        current_frame = current_frame + 1 
        print("current_frame", current_frame)
        frame_gray = cv.cvtColor(frame,cv.COLOR_BGR2GRAY)
                    
        
        particles = predict(particles, velocity, std, frame, w, h)        
        
        weights = update(particles, frame, hist_roi_norm, w, h) 
        
        obj = np.int32(estimate(particles, weights))       
       
        velocity[0] = obj[0] - bb[0]
        velocity[1] = obj[1] - bb[1]            
        print('velocity = ', velocity)
        bb = obj.copy()
        
        indexes = resample(weights)
        particles, weights = resample_from_index(particles, weights, indexes)               
        
        for i in range(N):            
            img2 = cv.rectangle(frame, (np.int32(particles[i,0]), np.int32(particles[i,1])), (np.int32(particles[i,0]) + w - 1, np.int32(particles[i,1]) + h - 1), (0,255,0), 1)
        
        img2 = cv.rectangle(frame, (obj[0], obj[1]), (obj[0] + w - 1, obj[1] + h - 1), (0, 255, 255), 4)
        cv.imshow('img2', img2)   
        cv.waitKey(1)
        if current_frame > max_number_of_frame_to_run:
            break
            
        if cv.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break

# after playing the video, release the video capture    
cap.release()                
# close all the frames
cv.destroyAllWindows()

current_frame 1
velocity =  [0, 1]
current_frame 2
velocity =  [0, 0]
current_frame 3
velocity =  [0, 1]
current_frame 4
velocity =  [0, -1]
current_frame 5
velocity =  [0, 1]
current_frame 6
velocity =  [0, -1]
current_frame 7
velocity =  [0, 1]
current_frame 8
velocity =  [1, -1]
current_frame 9
velocity =  [0, 1]
current_frame 10
velocity =  [0, 0]
current_frame 11
velocity =  [0, 0]
current_frame 12
velocity =  [0, 0]
current_frame 13
velocity =  [0, 0]
current_frame 14
velocity =  [-1, 0]
current_frame 15
velocity =  [1, 0]
current_frame 16
velocity =  [0, 0]
current_frame 17
velocity =  [0, 0]
current_frame 18
velocity =  [0, -1]
current_frame 19
velocity =  [0, 0]
current_frame 20
velocity =  [0, 0]
current_frame 21
velocity =  [0, 0]
current_frame 22
velocity =  [0, 0]
current_frame 23
velocity =  [0, 0]
current_frame 24
velocity =  [0, 0]
current_frame 25
velocity =  [0, 0]
current_frame 26
velocity =  [0, 0]
current_frame 27
velocity =  [0, 0]
current_frame 28
velocity =  [0,

In [21]:
cv.destroyAllWindows()